In [1]:
import pandas as pd

In [2]:
# Exercise 1: DataFrame Manipulation
# Given the following DataFrame, normalize the 'Revenue' 
# column so that all values are between 0 and 1
# NewValue = (((OldValue - OldMin) * (NewMax - NewMin)) / (OldMax - OldMin)) + NewMin
# Here NewMin = 0 and NewMax = 1

df = pd.DataFrame({
    'Company': ['A', 'B', 'C', 'D'],
    'Revenue': [25000, 47000, 53000, 32000]
})
# OldMin = df.Revenue.min()
OldMin = df['Revenue'].min() # method 1
OldMax = pd.Series.max(df.Revenue) # method 2
df['Revenue_Normalized'] = [(((OldValue - OldMin) * (1 - 0)) / (OldMax - OldMin)) + 0 for OldValue in df['Revenue']]
print(df)

  Company  Revenue  Revenue_Normalized
0       A    25000            0.000000
1       B    47000            0.785714
2       C    53000            1.000000
3       D    32000            0.250000


In [3]:
df = pd.DataFrame({
    'Company': ['A', 'B', 'C', 'D'],
    'Revenue': [25000, 47000, 53000, 32000]
})
# OldMin = df.Revenue.min()
for i in df.columns:
    if i.type == int:
        print i
print(df)

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(...)? (3073746064.py, line 8)

In [3]:
# Exercise 2: Custom Aggregations
# Given the DataFrame below, calculate the sum of Revenue for each company.

df = pd.DataFrame({
    'Company': ['A', 'B', 'A', 'B', 'A'],
    'Revenue': [20000, 35000, 23000, 48000, 12000]
})
gk = df.groupby('Company')
result = gk['Revenue'].aggregate('sum')
print(result)

Company
A    55000
B    83000
Name: Revenue, dtype: int64


In [4]:
# Exercise 3: Merge Two DataFrames Using Different Join Types
# Perform an outer join between two DataFrames.
df1 = pd.DataFrame({'ID': [1, 2, 3], 'Name': ['Alice', 'Bob', 'Charlie']})
df2 = pd.DataFrame({'ID': [2, 3, 4], 'City': ['New York', 'Paris', 'Tokyo']})
res  = pd.merge(df1, df2, on='ID', how='outer')
print(res)

   ID     Name      City
0   1    Alice       NaN
1   2      Bob  New York
2   3  Charlie     Paris
3   4      NaN     Tokyo


In [5]:
# Exercise 4: Identifying and Removing Duplicate Entries Efficiently
# while retaining the last transaction for each customer
data = {
    'customer_id': [1, 2, 2, 3, 3, 3],
    'transaction_id': [101, 102, 103, 104, 105, 106],
    'date': pd.to_datetime(['2025-04-01', '2025-04-02', '2025-04-03', '2025-04-01', '2025-04-02', '2025-04-03'])
}
df = pd.DataFrame(data)
df1 = df.groupby('customer_id')
result = df1['date'].aggregate('max')
# df2 = df.groupby('customer_id')['date'].aggregate('max')
# df3 = df.groupby('customer_id')['date'].max()
# df4 = df.groupby('customer_id').aggregate({'date': 'max'})
print(result)

customer_id
1   2025-04-01
2   2025-04-03
3   2025-04-03
Name: date, dtype: datetime64[ns]


In [6]:
# Method 2:
df_cleaned = df.sort_values(by=['date']).drop_duplicates(subset=['customer_id'], keep='last')
print(df_cleaned)

   customer_id  transaction_id       date
0            1             101 2025-04-01
2            2             103 2025-04-03
5            3             106 2025-04-03


In [18]:
# Exercise 5: Handeling Missing Data with Advanced Imputation Techniques
# Scenario: You have a CSV file named sales_data.csv containing daily sales records.
# Your task is to read the file, clean the data, and generate insights such as total revenue, 
# top-selling products and average daily sales.
# Data Cleaning: Handling missing values to ensure accuracy.
# Revenue Calculation: Computing total sales value per transaction.
# Best Seller Identification: Grouping and summarizing top-performing products.
# Trend Analysis: Determining the daily sales trend.

df_original = pd.read_csv("Datasets/sales_data.csv")
df = df_original.copy()

print(df.head())
print(df.info()) # Check for missing values and data types

print("Missing values before cleaning:\n")
print(df.isnull().sum())

df.fillna(df.mean(numeric_only=True), inplace=True)

# Fill missing values in categorical (object) columns with the mode (most frequent value)
for col in df.columns:
    if df[col].dtype == 'object' and df[col].isnull().any():  # Make sure there's at least one missing value
        df[col] = df[col].fillna(df[col].mode()[0])

# for col in df.select_dtypes(include='object').columns:
#     if df[col].isnull().any():
#         df[col].fillna(df[col].mode()[0], inplace=True)

print("Missing values after cleaning up:\n")
print(df.isnull().sum())

# Calculate revenue for each row: quantity * price
# Make sure 'Order_Quantity' and 'Unit_Price' exist and are numeric
if 'Order_Quantity' in df.columns and 'Unit_Price' in df.columns:
    df['revenue'] = df['Order_Quantity'] * df['Unit_Price']
else:
    raise ValueError("Missing 'Order_Quantity' or 'Unit_Price' columns for revenue calculation")

total_revenue = df['revenue'].sum()
print(f"Total Revenue: ${total_revenue:.2f}")

if 'Product' in df.columns:
    best_sellers = df.groupby('Product')['revenue'].sum().sort_values(ascending=False)
    print("Top 10 Best Selling Products:\n" + str(best_sellers.head(10))) # by default it is 5.
else:
    raise ValueError("Missing 'Product' column for best-selling products analysis")

daily_series = df.groupby('Date')['revenue'].sum().mean()
print(f"Average daily sales revenue: ${daily_series:.2f}")

         Date  Day     Month  Year  Customer_Age       Age_Group  \
0  2013-11-26   26  November  2013            19     Youth (<25)   
1  2015-11-26   26  November  2015            19     Youth (<25)   
2  2014-03-23   23     March  2014            49  Adults (35-64)   
3  2016-03-23   23     March  2016            49  Adults (35-64)   
4  2014-05-15   15       May  2014            47  Adults (35-64)   

  Customer_Gender    Country             State Product_Category Sub_Category  \
0               M     Canada  British Columbia      Accessories   Bike Racks   
1               M     Canada  British Columbia      Accessories   Bike Racks   
2               M  Australia   New South Wales      Accessories   Bike Racks   
3               M  Australia   New South Wales      Accessories   Bike Racks   
4               F  Australia   New South Wales      Accessories   Bike Racks   

               Product  Order_Quantity  Unit_Cost  Unit_Price  Profit  Cost  \
0  Hitch Rack - 4-Bike         